In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/Installment/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(17015, 2)


,label,split_text
0,1,还 没
1,0,然后 我 想到 一种 借用 的 方式
2,1,我 没有 多少 钱
3,1,你 能 减少 一些 吗 ？
4,0,我 可以 回来
5,1,优惠 太低
6,1,怎么 会 有 这么 多钱 ？
7,0,也
8,2,没有 钱 可以 解救 。
9,0,你 信息 我


In [3]:
data.split_text.iloc[42]

'我 的 家人 很穷'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<17015x20934 sparse matrix of type '<class 'numpy.float64'>'
	with 122385 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.911078460182


In [6]:
print(clf.predict(phrase))

[1 0 1 ..., 0 0 2]


# logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.87352336174


In [9]:
print(log_r.predict(phrase))

[1 0 1 ..., 0 0 2]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([1, 0, 1, ..., 0, 0, 2])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([1, 0, 1, ..., 0, 0, 2])

In [23]:
params = {
    'learning_rate': 0.2,
    'num_iterations':10000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.261769
[10]	valid_0's multi_error: 0.254187
[15]	valid_0's multi_error: 0.243315
[20]	valid_0's multi_error: 0.234852
[25]	valid_0's multi_error: 0.227505
[30]	valid_0's multi_error: 0.222274
[35]	valid_0's multi_error: 0.217338
[40]	valid_0's multi_error: 0.211637
[45]	valid_0's multi_error: 0.204761
[50]	valid_0's multi_error: 0.198883
[55]	valid_0's multi_error: 0.196121
[60]	valid_0's multi_error: 0.192183
[65]	valid_0's multi_error: 0.189186
[70]	valid_0's multi_error: 0.185425
[75]	valid_0's multi_error: 0.182427
[80]	valid_0's multi_error: 0.180488
[85]	valid_0's multi_error: 0.178607
[90]	valid_0's multi_error: 0.17749
[95]	valid_0's multi_error: 0.175375
[100]	valid_0's multi_error: 0.173905
[105]	valid_0's multi_error: 0.171848
[110]	valid_0's multi_error: 0.170379
[115]	valid_0's multi_error: 0.168146
[120]	valid_0's multi_error: 0.167029
[125]	valid_0's multi_error: 0.16509
[130]	val

[1125]	valid_0's multi_error: 0.121481
[1130]	valid_0's multi_error: 0.121481
[1135]	valid_0's multi_error: 0.121481
[1140]	valid_0's multi_error: 0.121481
[1145]	valid_0's multi_error: 0.121481
[1150]	valid_0's multi_error: 0.121481
[1155]	valid_0's multi_error: 0.121481
[1160]	valid_0's multi_error: 0.121481
[1165]	valid_0's multi_error: 0.121481
[1170]	valid_0's multi_error: 0.121481
[1175]	valid_0's multi_error: 0.121481
[1180]	valid_0's multi_error: 0.121481
[1185]	valid_0's multi_error: 0.121481
[1190]	valid_0's multi_error: 0.121481
[1195]	valid_0's multi_error: 0.121481
[1200]	valid_0's multi_error: 0.121481
[1205]	valid_0's multi_error: 0.121481
[1210]	valid_0's multi_error: 0.121481
[1215]	valid_0's multi_error: 0.121481
[1220]	valid_0's multi_error: 0.121481
[1225]	valid_0's multi_error: 0.121481
[1230]	valid_0's multi_error: 0.121481
[1235]	valid_0's multi_error: 0.121481
Early stopping, best iteration is:
[1037]	valid_0's multi_error: 0.121422


In [24]:
print(lgbm_model.predict(phrase))

[[  3.12533263e-01   5.55487585e-01   1.17354883e-01]
 [  9.37430438e-01   4.56259241e-02   1.61538680e-02]
 [  1.04141671e-03   8.88127882e-01   1.13833463e-01]
 ..., 
 [  9.89032234e-01   1.14928956e-02   5.96283143e-03]
 [  9.95708798e-01   8.89596444e-03   6.29798729e-04]
 [  1.87720279e-01   1.28159225e-01   6.14914362e-01]]


In [25]:
phrase

<17015x20934 sparse matrix of type '<class 'numpy.float64'>'
	with 122385 stored elements in Compressed Sparse Row format>

# Save

In [19]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/Installment/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/Installment/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/Installment/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/Installment/Lgbm.pickle", "wb"))

# Test

In [70]:
import jieba
sentence = '我也没有啊'
sentence = jieba.cut(sentence, cut_all = False)
sentence = ' '.join(sentence)
test = phrase_vectorizer.transform([sentence])

In [71]:
clf.predict_proba(test) # linear svc

array([[ 0.2227147 ,  0.76185917,  0.01542612]])

In [72]:
log_r.predict_proba(test) # logistic

array([[ 0.4645298 ,  0.45500487,  0.08046533]])

In [73]:
lgbm_model.predict(test) # light gbm

array([[ 0.8144944 ,  0.14162374,  0.00217648]])

# Labeling

In [24]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.998920196558


# Save the whole thing

In [25]:
# import jieba
# import numpy as np

# class Installment:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [26]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
clf = pickle.load(open("../../savedModel/Installment/LinearSVC.pickle", "rb"))
lgbm_model = pickle.load(open("../../savedModel/Installment/Lgbm.pickle", "rb"))
phrase_vectorizer = pickle.load(open("../../savedModel/Installment/tfidf.pickle", "rb"))
log_r = pickle.load(open("../../savedModel/Installment/Logistic.pickle", "rb"))

In [3]:
from Installment_py import Installment
ifk = Installment(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer,) #jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/Installment/Installment.pickle", "wb"))

(1, [1, 0.84518257422120568])